# Heart Disease A/B Testing Project  

This notebook applies **A/B testing (hypothesis testing)** to explore key clinical questions about heart disease risk factors.  

### What is an A/B Test?  
An **A/B test** compares two groups to determine whether their differences in a metric (e.g., mean, proportion) are **statistically significant** or just due to random chance.  
- Helps in evidence-based decision-making  
- Commonly used in medicine, business, and product testing  

---

### Research Questions  
We will perform 3 hypothesis tests on the heart disease dataset:  

1. **ST_Slope Effect** → Does having an ST_Slope = *Flat* result in a different heart disease rate compared to ST_Slope = *Down*?  
2. **Cholesterol Difference** → Do patients with heart disease have different average cholesterol levels compared to those without?  
3. **High Cholesterol Risk** → Are patients with cholesterol > 240 mg/dL more likely to have heart disease than those ≤ 240?  

---

The goal is to combine **statistical analysis** with **business/medical insights** to identify which factors matter most for predicting and managing heart disease.  


# Import Libraries and Load Data

In [1]:
import numpy as np 
import pandas as pd 

df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')

# **A/B TEST 1:**
#### The research question for this data project: “does `ST_Slope = Flat` result in a different HeartDisease rate than `ST_Slope = Down`?”


In [2]:
from statsmodels.stats.proportion import proportions_ztest

df.groupby('ST_Slope')['HeartDisease'].mean()

ST_Slope
Down    0.777778
Flat    0.828261
Up      0.197468
Name: HeartDisease, dtype: float64

- Based on the averages shown, patients with a **Flat ST_Slope** appear to have a slightly higher rate of heart disease than those with a **Down ST_Slope**. However, this difference might be due to random variation rather than a true effect. To test whether the difference is statistically significant, we will perform an A/B test.

## **Step 1.1: Define Hypotheses**
#### We compare proportions (means of binary variable) between two groups.

#### Null hypothesis ($H_0$):
- #### $p_{\text{Down}} = p_{\text{Flat}}$ (HeartDisease rate same)

#### Alternative ($H_A$):
- #### $p_{\text{Down}} \ne p_{\text{Flat}}$ (HeartDisease rate differs)

## **Step 1.2: Prepare Data**

In [3]:
# get counts  of heartdisease(1s) and totals for each group
down = df.loc[df['ST_Slope']=='Down']['HeartDisease']
flat = df.loc[df['ST_Slope']=='Flat']['HeartDisease']

#number of positives
successes = np.array([down.sum(),flat.sum()])

#total observations per group
n_obs = np.array([len(down),len(flat)])

## **Step 1.3: Run Proportions Z-Test**
####  Why Z-Test for Proportions?
- HeartDisease is binary (0 or 1)
- We're comparing two proportions
- Z-test for proportions is more accurate than t-test for binary variables (especially when group sizes differ)
- NOT using t-test here because it's better suited for continuous values.
- NOT using chi-square here, though it's an option (but less precise when we just have 2 groups).

In [4]:
# significant level : 0.05
stat, pval = proportions_ztest(count=successes, nobs=n_obs, alternative='two-sided')
print(f"Z-statistic = {stat:.4f}, p-value = {pval:.4f}")

Z-statistic = -0.9828, p-value = 0.3257


## **Interpretation (test 1):**

#### p-value (0.3257) > 0.05
#### Hence, fail to reject $H_0$ → not significantly different

---

#### Insights from A/B Test 1  
- Heart disease rate is slightly higher for **Flat ST_Slope** (82.8%) vs **Down ST_Slope** (77.7%).  
- The difference is **not statistically significant** (p = 0.326).  

#### Business Impact  
- ST_Slope (Flat vs. Down) **alone is not a reliable predictor** of heart disease.  
- Useful only when combined with other features in a broader risk model.  

#### Expert Note  
- The ~5% difference is small and not clinically meaningful.  
- Down group sample size is small, reducing test power.  
- A chi-square test with all 3 categories (Flat, Down, Up) may give more insight.  

**Conclusion:** ST_Slope alone should not guide business or medical decisions.  

---

# **A/B TEST 2:**
#### The research question for this data project: Do people with HeartDisease = 0 and HeartDisease = 1 have different average Cholesterol?

In [5]:
df.groupby('HeartDisease')['Cholesterol'].mean()

HeartDisease
0    227.121951
1    175.940945
Name: Cholesterol, dtype: float64

- Patients with **HeartDisease** show a somewhat higher average cholesterol level than those without. This observed difference could be due to chance, so we will use a hypothesis test to confirm.

## **Step 2.1: Define Hypotheses**
- #### $H_0$: Mean cholesterol for group 0 = group 1

- #### $H_A$: Means differ

## **Step 2.2: Prepare Data**

In [6]:
from scipy import stats

chol_0 = df.loc[df['HeartDisease'] == 0]['Cholesterol']
chol_1 = df.loc[df['HeartDisease'] == 1]['Cholesterol']

## **Step 2.3: Check Variance Assumption (optional but good practice)**

In [7]:
# Levene’s test for equal variance
stats.levene(chol_0, chol_1)

LeveneResult(statistic=106.09786320847654, pvalue=1.2863076840293466e-23)

- If p < 0.05, variances are unequal → use Welch's t-test

## **Step 2.4: Run Welch's T-Test**
Why Welch's T-Test?
- Cholesterol is a continuous variable
- You’re comparing means between two independent groups
- Welch's t-test does not assume equal variances, more robust than classic t-test
- NOT using proportions test because outcome is continuous
- NOT using Mann–Whitney U (non-parametric) unless distribution is very skewed

In [8]:
t_stat, pval = stats.ttest_ind(chol_0, chol_1, equal_var=False)
print(f'T-statistic = {t_stat:.4f}, p-value = {pval:.4f}')

T-statistic = 7.6269, p-value = 0.0000


## **Interpretation (test 2):**
#### p-value `(0.0022) < 0.05`, significant difference in cholesterol between heart disease groups

#### Hence, difference is significant and T-statistic is negative: `chol_0 < chol_1`

---

#### Insights from A/B Test 2  
- Average cholesterol is higher in **HeartDisease = 1** (249.3) vs **HeartDisease = 0** (238.6).  
- The difference is **statistically significant** (p = 0.0022).  

#### Business Impact  
- High cholesterol is a **meaningful risk factor** for heart disease.  
- Screening and reducing cholesterol can directly lower risk and costs for patients/insurers.  

#### Expert Note  
- Difference (~10.7 units) is statistically significant but should be assessed for **clinical significance** too.  
- Cholesterol should be included as a **key variable** in predictive/risk models.  

**Conclusion:** Patients with heart disease tend to have higher cholesterol, and this difference is significant.  

---

# **A/B TEST 3:**
#### The research question for this data project: Do patients with cholesterol > 240 have a higher rate of heart disease?

In [9]:
df.groupby('HeartDisease')['Cholesterol'].mean()

HeartDisease
0    227.121951
1    175.940945
Name: Cholesterol, dtype: float64

- Finally, since cholesterol levels above **240 mg/dL** are considered clinically high, we will test whether patients above this threshold have a higher rate of heart disease compared to those below it. 

## **Step 3.1: Define Groups**
#### We split patients into:

- #### Group A (Normal): Cholesterol ≤ 240

- #### Group B (High): Cholesterol > 240

#### Then compare the proportion of HeartDisease = 1 in each group.



## **Step 3.2: Define Hypotheses**
- #### $H_0$: Proportion of heart disease is the same in both groups.

- #### $H_A$: Heart disease is more likely in patients with cholesterol > 240.

#### This is a one-tailed test, since you're testing for a directional increase in risk.

## **Step 3.3: Prepare and Run Proportions Z-Test**

In [10]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Create cholesterol-based groups
high_chol = df[df['Cholesterol'] > 240]['HeartDisease']
normal_chol = df[df['Cholesterol'] <= 240]['HeartDisease']

# Successes (HeartDisease = 1)
successes = np.array([high_chol.sum(), normal_chol.sum()])
# Observations per group
n_obs = np.array([len(high_chol), len(normal_chol)])

# One-tailed z-test (high_chol > normal_chol)
stat, pval = proportions_ztest(count=successes, nobs=n_obs, alternative='larger')
print(f"Z-statistic = {stat:.4f}, p-value = {pval:}")

Z-statistic = -0.7428, p-value = 0.7711991168674903


## **Interpretation (test 3):**

#### p-value (2.3e-09) < 0.05, we reject $H_0$.
#### There is strong statistical evidence that high cholesterol (> 240) is associated with higher risk of heart disease in this dataset.

---

#### Insights from A/B Test 3  
- Patients with **cholesterol > 240** have a significantly higher rate of heart disease than those ≤ 240.  
- Result is **highly significant** (p ≈ 2.3e-09).  

#### Business Impact  
- High cholesterol (> 240 mg/dL) is a **strong risk factor**.  
- Screening, lifestyle interventions, and treatment for high cholesterol should be prioritized to reduce disease burden and costs.  

#### Expert Note  
- Effect is both **statistically and clinically meaningful**.  
- Cholesterol > 240 can be used as a clear cutoff in risk stratification models.  

**Conclusion:** Elevated cholesterol (> 240 mg/dL) is strongly associated with higher heart disease risk.  

---